In [129]:
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image as tfimg
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import RandomTranslation
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [28]:
notesList = [
("100a", "100_awers.jpg"),
("100r", "100_rewers.jpg"),
("10a", "10_awers.jpg"),
("10r", "10_rewers.jpg"),
("200a", "200_awers.jpg"),
("200r", "200_rewers.jpg"),
("20a", "20_awers.jpg"),
("20r", "20_rewers.jpg"),
("50a", "50_awers.jpg"),
("50r", "50_rewers.jpg"),
("100a", "NBP_500_strona_przednia2.jpg"),
("100r", "tyl_100.jpg")
]

In [29]:
def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)
    flipped = tf.image.flip_left_right(image)
    visualize(image, flipped)

In [30]:
def create_data_augmentation_model():
    data_augmentation = tf.keras.Sequential([
        RandomZoom(height_factor=0.2, width_factor=0.2, fill_mode="constant"),
        RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="constant"),
        RandomRotation(0.2)
    ])
    return data_augmentation

In [31]:
def load_notes_images(directory, notesList):
    result = []
    for pair in notesList:
        label = pair[0]
        fullPath = directory + '/' + pair[1]
        rawImage = tfimg.load_img(fullPath, target_size=(224, 224))
        imgArr = tfimg.img_to_array(rawImage)
        expanded = np.expand_dims(imgArr, axis=0)
        x = preprocess_input(expanded)
        result.append((label, fullPath, rawImage, expanded, x))    
    return result

In [67]:
def create_augmented_data(notesImages, data_augmentation):
    augmentedData = []
    for imageTuple in notesImages:
        label, fullPath, rawImage, expanded, preprocessed = imageTuple
        augmentedData.append(imageTuple)
        for i in range(50):
            augmented_image = data_augmentation(expanded)
            preprocessed_augmented = preprocess_input(augmented_image)
            augmentedData.append((label, "", None, augmented_image, preprocessed_augmented))
    return augmentedData

In [33]:
def load_model():
    model = VGG16(weights='imagenet', include_top=False)
    return model

In [34]:
def extract_features(augmentedData, model):
    extractedFeatures = []

    for ad in augmentedData:
        preprocessedImage = ad[4]
        label = ad[0]
        features = model.predict(preprocessedImage)
        extractedFeatures.append((label, features))
        
    return extractedFeatures

In [35]:
notesImages = load_notes_images("d:/Dane/banknoty_PLN", notesList)

In [68]:
data_augmentation = create_data_augmentation_model()

In [69]:
augmentedData = create_augmented_data(notesImages, data_augmentation)

In [70]:
random.shuffle(augmentedData)

In [71]:
model = load_model()

In [72]:
extractedFeatures = extract_features(augmentedData, model)

In [73]:
def convert_extracted_features_to_np_array(extractedFeatures):
    acc = None
    labels = []

    for ef in extractedFeatures:
        aux = ef[1].flatten().reshape((1, 25088))
        label = ef[0]
        if acc is None:
            acc = aux
        else:
            acc = np.append(acc, aux, axis=0)
        labels.append(label)
        
    labelsArr = np.array(labels)
            
    return (acc, labelsArr)

In [120]:
merged_features = convert_extracted_features_to_np_array(extractedFeatures)

In [130]:
def create_and_evaluate_model(merged_features, componentsCount):
    X = merged_features[0]
    y = merged_features[1]    
    
    min_max_scaler = preprocessing.MinMaxScaler()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X_train_minmax = min_max_scaler.fit_transform(X_train)
    
    pca = PCA(n_components=componentsCount)
    X_train_minmax_pcs = pca.fit_transform(X_train_minmax)
    
    X_test_minmax = min_max_scaler.transform(X_test)
    X_test_minmax_pca = pca.transform(X_test_minmax)
    
    X_train_preprocessed = X_train_minmax_pcs
    X_test_preprocessed = X_test_minmax_pca
    
    nextModel = GridSearchCV(estimator=SVC(), param_grid={'C': [0.1, 1, 10, 100], 'kernel': ('linear', 'rbf')}) #SVC(gamma='auto')
    
    readySvcModel = nextModel.fit(X_train_preprocessed, y_train)
    y_predict = nextModel.predict(X_test_preprocessed)
    
    acc = accuracy_score(y_test, y_predict)
    return acc

In [131]:
create_and_evaluate_model(merged_features, 100)

0.9702970297029703

In [132]:

#plt.figure(figsize=(10, 10))


#plt.imshow(image[0])

#for i in range(10):
#    augmented_image = augmentedData[i][3]
#    print(augmented_image.shape)
#    ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(augmented_image[0])
#   plt.axis("off")